In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !%cd /content/drive/MyDrive/zeroshot_new_paper

In [36]:
# import sys
# sys.path.insert(0,'/content/drive/MyDrive/zeroshot_new_paper')

Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


# === KeyZeroBERTo ===

# Dependencies

In [37]:
# !pip install transformers
# !pip install keybert==0.5.0
# !pip install datasets
# !pip install flair|
# %env PYTORCH_ENABLE_MPS_FALLBACK=1

In [38]:
# !pip install pandarallel

In [39]:
# !pip install evaluate

In [193]:
%load_ext autoreload
%autoreload 2
import nltk
import ipywidgets
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from transformers import pipeline
import torch
import datetime
import os

import datasets_handler
import evaluation_metrics
import zeroberto
# def getAgora():
#     return str(datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S"))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Data Prep

## Source

In [255]:
# which_dataset = 'folhauol' 
# which_dataset = 'bbc-news'
which_dataset = 'ag_news'

hyp_template = "O tema principal deste texto é {}."
# hyp_template = "this text is about {}."
# hyp_template = "this article is about {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)


Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

## Dataset build

### Train Data

In [253]:
# arq_train = '/content/drive/MyDrive/zeroshot_new_paper/BBC News Train.csv'
# train = pd.read_csv(arq_train)
train = raw_data
len(train),train.columns
# train = raw_data[~(raw_data['full_text'].isnull())].copy()
# train = raw_data[~(raw_data['full_text'].isnull())].sample(max_inferences,random_state=42).copy()

(92241,
 Index(['Unnamed: 0', 'title', 'text', 'date', 'category', 'subcategory',
        'link', 'data', 'full_text'],
       dtype='object'))

In [254]:
labels_list = list(train[class_col].unique())
print(labels_list)

['Poder e Política no Brasil', 'Mercado', 'Notícias de fora do Brasil', 'Esporte', 'Tecnologia', 'Meio Ambiente', 'Equilíbrio e Saúde', 'Educação', 'TV, Televisão e Entretenimento', 'Ciência', 'Turismo', 'Comida']


# Sentence Extraction

In [183]:
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()

In [184]:
try:
    print(train['sentences'])
except:
    train['sentences'] = train[data_col].progress_apply(lambda txt:tokenizer.tokenize(txt))
    train.head(2)

100%|██████████| 127600/127600 [00:02<00:00, 44911.20it/s]


In [186]:
sentences_to_classify = []
for sentences in train['sentences']:
  try:
    for sentence in sentences:
      sentences_to_classify.append(sentence)
      break
  except:
    print(train['sentences'])
train['sentence_to_zeroshot'] = sentences_to_classify
len(train)


127600

In [188]:
train['len_sentences'] = train['sentences'].progress_apply(lambda txt:len(txt))
# train[['len_sentences']].describe()

100%|██████████| 127600/127600 [00:00<00:00, 1950867.51it/s]


# Zero-Shot

## Data

In [207]:
# max_inferences = len(train) #estava em 5000
max_inferences = 5000

# zeroshot_method = "probability_threshold"
probability_goal = 0.9

zeroshot_method = "top_n_goal"  
top_n_goal = 8

zeroshot_config = {
    'dataset':which_dataset,
    'method':zeroshot_method,
    'prob_goal':probability_goal,
    'top_n_goal':top_n_goal,
    'max_inferences':max_inferences,
    'labels':labels_list,#list(dict_classes_folha.values())
    'template': hyp_template,
    'random_state':422
}
zeroshot_config

{'dataset': 'ag_news',
 'method': 'top_n_goal',
 'prob_goal': 0.9,
 'top_n_goal': 8,
 'max_inferences': 5000,
 'labels': ['business', 'science and technology', 'sports', 'world'],
 'template': 'this text is about {}.',
 'random_state': 422}

In [208]:
zeroshot_data = train.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state'])
len(zeroshot_data)

5000

## Model

In [231]:
# classifier = pipeline("zero-shot-classification",
#                       model="joeddav/xlm-roberta-large-xnli",device="cpu") # device = 'cuda'
classifier = zeroberto.ZeroBERTo(classes_list=zeroshot_config['labels'],hypothesis_template=zeroshot_config['template'])

## Inference

In [232]:
%%time
# zeroshot_results = zeroberto.runZeroShotPipeline(classifier,zeroshot_data['sentence_to_zeroshot'],zeroshot_config)

zeroshot_results = zeroberto.runZeroberto(classifier,zeroshot_data[data_col],zeroshot_config)
# print(len(zeroshot_results))
### 100 predições ---> <240 segundos (4 min)
### 200 predições ---> <480 segundos (8 min)
### 1000 predições ---> <2400 segundos (40 min)
### 5000 predições ---> 12000 segundos (200 min) (3h 20min)

Preds: 50  - Total time: 2.35 seconds - ETA: 3.9 minutes
Preds: 100  - Total time: 4.54 seconds - ETA: 3.8 minutes
Preds: 150  - Total time: 6.73 seconds - ETA: 3.7 minutes
Preds: 200  - Total time: 8.75 seconds - ETA: 3.6 minutes
Preds: 250  - Total time: 10.95 seconds - ETA: 3.6 minutes
Preds: 300  - Total time: 13.31 seconds - ETA: 3.7 minutes
Preds: 350  - Total time: 15.48 seconds - ETA: 3.7 minutes
Preds: 400  - Total time: 17.63 seconds - ETA: 3.7 minutes
Preds: 450  - Total time: 19.69 seconds - ETA: 3.6 minutes
Preds: 500  - Total time: 21.76 seconds - ETA: 3.6 minutes
Preds: 550  - Total time: 24.08 seconds - ETA: 3.6 minutes
Preds: 600  - Total time: 26.28 seconds - ETA: 3.6 minutes
Preds: 650  - Total time: 28.52 seconds - ETA: 3.7 minutes
Preds: 700  - Total time: 30.75 seconds - ETA: 3.7 minutes
Preds: 750  - Total time: 32.88 seconds - ETA: 3.7 minutes
Preds: 800  - Total time: 35.23 seconds - ETA: 3.7 minutes
Preds: 850  - Total time: 37.44 seconds - ETA: 3.7 minutes
Pr

In [233]:
label_probabilities_df = pd.DataFrame(zeroshot_results,columns=zeroshot_config['labels'])

In [234]:
# label_probabilities_df = zeroberto.formatZeroShotResults(zeroshot_results)

In [235]:
def getTopLabel(row):
  return row.idxmax()
def getTopProb(row):
  return row.max()

label_results = label_probabilities_df.apply(getTopLabel,axis=1)
prob_results = label_probabilities_df.apply(getTopProb,axis=1)
label_results_df = pd.Series(label_results,name='prediction')

In [236]:
true_labels_df = raw_data[raw_data.index.isin(train.index)][class_col]
final_result_df = pd.concat([true_labels_df,label_results_df],axis=1)

In [237]:
vector_true = ( final_result_df[class_col].str.strip().str.lower() 
               == final_result_df['prediction'].str.strip().str.lower() )

## Metrics

In [238]:
final_result_df_encoded = evaluation_metrics.Encoder(final_result_df,['prediction',class_col])

In [239]:
# final_result_df_encoded['predicted?'] = False
# final_result_df_encoded.loc[0:len(zeroshot_results),'predicted?'] = True
# #

In [241]:
print(all_metrics)
all_metrics = evaluation_metrics.get_metrics(final_result_df_encoded['prediction_code'][0:len(zeroshot_results)].to_list(),
                               final_result_df_encoded[class_col+'_code'][0:len(zeroshot_results)].to_list())
print(all_metrics)

{'weighted': [{'accuracy': 0.2534}, {'precision': 0.2642732671609221}, {'recall': 0.2534}, {'f1': 0.2496590712523609}], 'macro': [{'accuracy': 0.2534}, {'precision': 0.2590240991448485}, {'recall': 0.25633417432813943}, {'f1': 0.24874448569928517}]}


### Probabilities

In [242]:
df_predictions_probs = pd.concat([final_result_df_encoded,
                                  pd.Series(prob_results,name='top_probability'),
                                  pd.Series(vector_true,name='right_wrong')],axis=1)

##### Top N results


In [243]:
def get_top_n_results(dataframe_results,top_n=1):
    df_top_n = dataframe_results.sort_values(['top_probability','prediction'], ascending=False).groupby('prediction').head(top_n)

    accuracy_top_n = df_top_n.right_wrong.sum()/len(df_top_n.right_wrong)

    accuracy = { "accuracy in top {}".format(top_n) : round(accuracy_top_n,3)}
    print(accuracy)
    return 

get_top_n_results(df_predictions_probs,top_n = 1)
get_top_n_results(df_predictions_probs,top_n = 2)
get_top_n_results(df_predictions_probs,top_n = 3)
get_top_n_results(df_predictions_probs,top_n = 4)
get_top_n_results(df_predictions_probs,top_n = 5)
get_top_n_results(df_predictions_probs,top_n = 6)
get_top_n_results(df_predictions_probs,top_n = 7)
get_top_n_results(df_predictions_probs,top_n = 8)
get_top_n_results(df_predictions_probs,top_n = 9)
get_top_n_results(df_predictions_probs,top_n = 10)
get_top_n_results(df_predictions_probs,top_n = 11)
get_top_n_results(df_predictions_probs,top_n = 12)
get_top_n_results(df_predictions_probs,top_n = 13)
get_top_n_results(df_predictions_probs,top_n = 14)
get_top_n_results(df_predictions_probs,top_n = 15)
get_top_n_results(df_predictions_probs,top_n = 16)
get_top_n_results(df_predictions_probs,top_n = len(zeroshot_results))

{'accuracy in top 1': 0.25}
{'accuracy in top 2': 0.25}
{'accuracy in top 3': 0.25}
{'accuracy in top 4': 0.25}
{'accuracy in top 5': 0.2}
{'accuracy in top 6': 0.25}
{'accuracy in top 7': 0.286}
{'accuracy in top 8': 0.25}
{'accuracy in top 9': 0.25}
{'accuracy in top 10': 0.25}
{'accuracy in top 11': 0.25}
{'accuracy in top 12': 0.25}
{'accuracy in top 13': 0.25}
{'accuracy in top 14': 0.268}
{'accuracy in top 15': 0.25}
{'accuracy in top 16': 0.234}
{'accuracy in top 5000': 0.253}


## Save Results

In [244]:
evaluation_metrics.saveZeroshotResults(zeroshot_config,df_predictions_probs)

predictions_and_probabilities_test_2023_02_13__03_16_31.csv
zeroshot_config_test_2023_02_13__03_16_31.csv


'2023_02_13__03_16_31'

# Load Previous Results

In [30]:
# df_test_results = pd.read_csv(r'/Users/alealcoforado/Documents/Projetos/Datasets/predictions_and_probabilities_test1.csv')

In [31]:
# df_test_results.index = df_test_results['Unnamed: 0']
# df_test_results.drop(columns="Unnamed: 0",inplace=True)